In [ ]:
%sql
DESCRIBE CATALOG `dev-catalog`;

info_name,info_value
Catalog Name,dev-catalog
Comment,
Owner,sachinsax_gmail.com#ext#@sachinsaxgmail.onmicrosoft.com
Catalog Type,Regular


In [ ]:
spark.sql("""USE CATALOG `dev-catalog`""")

DataFrame[]

In [ ]:
%sql
SELECT CURRENT_METASTORE();

current_metastore()
azure:eastus:50acce50-2335-4ca3-9327-b6dbbfed2c2d


In [ ]:
%sql
create schema if not exists `silver`

In [ ]:
%sql
create schema if not exists `gold`
managed location 'abfss://landing@databricksdevstg123.dfs.core.windows.net/landing'
-- this will create schema at abfss://landing@dbstoragesachin.dfs.core.windows.net/landing
-- check in catalog UI as well

In [ ]:
%sql
DESCRIBE EXTERNAL LOCATION `landing`

name,url,credential_name,owner,created_by,created_at,comment
landing,abfss://landing@databricksdevstg123.dfs.core.windows.net/,50acce50-2335-4ca3-9327-b6dbbfed2c2d-storage-credential-1735219246140,sachinsax_gmail.com#ext#@sachinsaxgmail.onmicrosoft.com,sachinsax_gmail.com#ext#@sachinsaxgmail.onmicrosoft.com,2024-12-26T13:27:53.082Z,


In [ ]:
landing_path = spark.sql("""DESCRIBE EXTERNAL LOCATION `landing`""").select("url").collect()[0][0]
# to extract url only from previous cell code, we have this one comman
print(landing_path)

abfss://landing@databricksdevstg123.dfs.core.windows.net/


In [ ]:
spark.sql(f"""create schema if not exists `test_gold_not_actual333` managed location '{landing_path}/test_loc2'""")

DataFrame[]

In [ ]:
dbutils.widgets.text(name="env",defaultValue="",label=" Enter the environment in lower case")
env = dbutils.widgets.get("env")

In [ ]:
landing_path2 = spark.sql("""DESCRIBE EXTERNAL LOCATION `landing`""").select("url").collect()[0][0]

In [ ]:
print(landing_path2)

abfss://landing@databricksdevstg123.dfs.core.windows.net/


In [ ]:
def create_landing_Schema(environment,path):
    print(f'Using {environment}_Catalog ')
    spark.sql(f""" USE CATALOG '{environment}-catalog'""")
    print(f'Creating Bronze Schema in {environment}_Catalog')
    spark.sql(f"""CREATE SCHEMA IF NOT EXISTS `Sachin_bronze` MANAGED LOCATION '{path}/bronze'""")
    print("************************************")

In [ ]:
create_landing_Schema(env,landing_path2)

# this will give output
# Using dev_Catalog
# Creating Bronze Schema in dev_Catalog
# ************************************

Using dev_Catalog 
Creating Bronze Schema in dev_Catalog
************************************


In [ ]:
def createTable_rawTraffic(environment):
    print(f'Creating raw_Traffic table in {environment}-catalog')
    spark.sql(f"""CREATE TABLE IF NOT EXISTS `{environment}-catalog`.`bronze`.`raw_traffic`
                        (
                            Record_ID INT,
                            Count_point_id INT,
                            Direction_of_travel VARCHAR(255),
                            Year INT,
                            Count_date VARCHAR(255),
                            hour INT,
                            Region_id INT,
                            Region_name VARCHAR(255),
                            Local_authority_name VARCHAR(255),
                            Road_name VARCHAR(255),
                            Road_Category_ID INT,
                            Start_junction_road_name VARCHAR(255),
                            End_junction_road_name VARCHAR(255),
                            Latitude DOUBLE,
                            Longitude DOUBLE,
                            Link_length_km DOUBLE,
                            Pedal_cycles INT,
                            Two_wheeled_motor_vehicles INT,
                            Cars_and_taxis INT,
                            Buses_and_coaches INT,
                            LGV_Type INT,
                            HGV_Type INT,
                            EV_Car INT,
                            EV_Bike INT,
                            Extract_Time TIMESTAMP
                    );""")

    print("************************************")

In [ ]:
def createTable_rawRoad(environment):
    print(f'Creating raw_roads table in {environment}-catalog')
    spark.sql(f"""CREATE TABLE IF NOT EXISTS `{environment}-catalog`.`bronze`.`raw_roads`
                        (
                            Road_ID INT,
                            Road_Category_Id INT,
                            Road_Category VARCHAR(255),
                            Region_ID INT,
                            Region_Name VARCHAR(255),
                            Total_Link_Length_Km DOUBLE,
                            Total_Link_Length_Miles DOUBLE,
                            All_Motor_Vehicles DOUBLE
                    );""")

    print("************************************")

In [ ]:
create_landing_Schema(env,landing_path2)
createTable_rawTraffic(env)
createTable_rawRoad(env)

Using dev_Catalog 
Creating Bronze Schema in dev_Catalog
************************************
Creating raw_Traffic table in dev-catalog
************************************
Creating raw_roads table in dev-catalog
************************************


### Part 0 End here